In [3]:
import boto3
from boto3.session import Session 
import json

session = Session(profile_name='bedrock',region_name='us-east-1')
client = session.client('bedrock-runtime')

model_id = 'anthropic.claude-instant-v1'

body ={
    'prompt': '\n\nHuman: 日本の春の魅力を教えてください。 \n\nAssistant:',
    'max_tokens_to_sample':300,
    'temperature':0.7,
    'top_p':0.9,
    'top_k':250
}

response = client.invoke_model(
    modelId=model_id,
    body=json.dumps(body),
    contentType='application/json',
    accept='application/json'
)

result = json.loads(response['body'].read())
print(result['completion'])


 はい、日本の春には以下のような魅力があります。

- 桜が満開になります。日本各地で花見イベントが開催され、桜は日本の春を代表するシンボルとなっています。

- 天気が快適になります。春先は日差しが強くなく、暖かい風が吹きます。

- 新緑が芽吹き始めます。緑の色が徐々に広がり、緑が満ちていきます。

- 花の種類が豊富になります。桜以外にも菜の花、椿、菜の花などの花が咲き誇ります。 

- 自然の生き物たちの活動が活発になります。鳥の鳴き声などがよく聞こえます。

- 田舎の緑や里山の景色が美しくなります。

- お盆までの行事が楽しめる


In [3]:
import boto3
from boto3.session import Session
import json

session = Session(profile_name='bedrock',region_name='us-east-1')
client = session.client('bedrock-runtime')

with open('input.txt','r',encoding='utf-8') as f:
    user_prompt = f.read().strip()

prompt = f"\n\nHuman: {user_prompt} \n\nAssistant:"

body = {
    'prompt': prompt,
    'max_tokens_to_sample': 800,
    'temperature': 0.7,
    'top_p': 0.9,
    'top_k': 250
}

response = client.invoke_model(
    modelId='anthropic.claude-instant-v1',
    body=json.dumps(body),
    contentType='application/json',
    accept='application/json'
)

result = json.loads(response['body'].read())
output_text = result['completion']

with open('output_txt','w',encoding='utf-8') as f:
    f.write(output_text)

print('出力が output_text に保存されました。')


出力が output_text に保存されました。


In [4]:
import boto3
import json
from boto3.session import Session

session = Session(profile_name='bedrock',region_name='us-east-1')
client = session.client('bedrock-runtime')
s3 = session.resource('s3')

bucket_name = 'test-backet-20250407'
input_key = 'prompt-test.txt'
output_key = 'result.txt'

obj = s3.Object(bucket_name,input_key)
user_prompt = obj.get()['Body'].read().decode('utf-8').strip()

prompt = f"\n\nHuman: {user_prompt} \n\nAssistant:"
body = {
    'prompt': prompt,
    'max_tokens_to_sample': 800,
    'temperature': 0.7,
    'top_p': 0.9,
    'top_k': 250
}

response = client.invoke_model(
    modelId='anthropic.claude-instant-v1',
    body=json.dumps(body),
    contentType='application/json',
    accept='application/json'
)

result = json.loads(response['body'].read())
answer = result['completion']

s3.Object(bucket_name,output_key).put(Body=answer.encode('utf-8'))

print('完了：S3に回答を保存しました。')

完了：S3に回答を保存しました。


In [4]:
#input.csvの内容をバッチ処理で回答させてoutput.csvに保存

import boto3
from boto3.session import Session
import json
import csv
import time

session = Session(profile_name='bedrock',region_name='us-east-1')
client = session.client('bedrock-runtime')

input_file = 'input.csv'
output_file = 'output.csv'

with open(input_file,'r',encoding='utf-8') as f:
    reader = csv.reader(f)
    prompts = [row[0] for row in reader if row]

with open(output_file,'w',encoding='utf-8',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['質問','回答'])

    for prompt_text in prompts:
        prompt = f'\n\nHuman: {prompt_text} \n\nAssistant:'

        body = {
            'prompt': prompt,
            'max_tokens_to_sample': 800,
            'temperature': 0.7,
            'top_p': 0.9,
            'top_k': 250
        }

        response = client.invoke_model(
            modelId='anthropic.claude-instant-v1',
            body=json.dumps(body),
            contentType='application/json',
            accept='application/json'
        )
        
        result = json.loads(response['body'].read())
        answer = result.get('completion','(回答無し)')

        writer.writerow([prompt_text,answer])
        print(f"完了：{prompt_text[:20]}... → 回答を保存")

        time.sleep(1)
        

完了：日本の魅力を教えてください。... → 回答を保存
完了：AWSの主要サービスを教えてください。... → 回答を保存
完了：ClaudeとchatGPTの違いを教え... → 回答を保存


In [2]:
import boto3
from boto3.session import Session
import json
import csv
import time

session = Session(profile_name='bedrock',region_name='us-east-1')
client = session.client('bedrock-runtime')

def run_claude(prompt_text):
    prompt = f"\n\nHuman: {prompt_text} \n\nAssistant:"
    body = {
        'prompt': prompt,
        'max_tokens_to_sample': 800,
        'temperature':0.7,
        'top_p': 0.9,
        'top_k': 250
    }

    response = client.invoke_model(
        modelId='anthropic.claude-instant-v1',
        body=json.dumps(body),
        contentType='application/json',
        accept='application/json'
    )

    result = json.loads(response['body'].read())
    return result.get('completion','(回答無し)')

input_file = 'input.csv'
output_file = 'output2.csv'

with open(input_file,'r',encoding='utf-8') as f:
    reader = csv.reader(f)
    prompts = [row[0] for row in reader if row]

with open(output_file,'w',encoding='utf-8',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['質問','回答'])

    for prompt_text in prompts:
        try:
            answer = run_claude(prompt_text)
            writer.writerow([prompt_text,answer])
            print(f"完了：{prompt_text[:20]}...→回答を保存")
            time.sleep(1)
        except Exception as e:
            print(f"エラー：{prompt_text[:20]}...→{e}")


完了：日本の魅力を教えてください。...→回答を保存
完了：AWSの主要サービスを教えてください。...→回答を保存
完了：ClaudeとchatGPTの違いを教え...→回答を保存


In [3]:
import boto3
from boto3.session import Session
import json
import csv
import time
from datetime import datetime

today_str = datetime.now().strftime('%Y-%m-%d')

session = Session(profile_name='bedrock',region_name='us-east-1')
client = session.client('bedrock-runtime')

def run_claude(prompt_text):
    prompt = f"\n\nHuman: {prompt_text} \n\nAssistant:"
    body = {
        'prompt': prompt,
        'max_tokens_to_sample': 800,
        'temperature':0.7,
        'top_p': 0.9,
        'top_k': 250
    }

    response = client.invoke_model(
        modelId='anthropic.claude-instant-v1',
        body=json.dumps(body),
        contentType='application/json',
        accept='application/json'
    )

    result = json.loads(response['body'].read())
    return result.get('completion','(回答無し)')

input_file = 'input.csv'
output_file = f'output_{today_str}.csv'

with open(input_file,'r',encoding='utf-8') as f:
    reader = csv.reader(f)
    prompts = [row[0] for row in reader if row]

with open(output_file,'w',encoding='utf-8',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['質問','回答'])

    for prompt_text in prompts:
        try:
            answer = run_claude(prompt_text)
            writer.writerow([prompt_text,answer])
            print(f"完了：{prompt_text[:20]}...→回答を保存")
            time.sleep(1)
        except Exception as e:
            print(f"エラー：{prompt_text[:20]}...→{e}")


完了：日本の魅力を教えてください。...→回答を保存
完了：AWSの主要サービスを教えてください。...→回答を保存
完了：ClaudeとchatGPTの違いを教え...→回答を保存


In [3]:
#Claudeにたいしてinput.csvの質問を送信して回答をoutput.csvに日付付きで出力するスクリプト

import boto3
import json
from boto3.session import Session
import csv
import time
from datetime import datetime

today_str = datetime.now().strftime('%Y-%m-%d')
input_file = 'input.csv'
output_file = f'output_{today_str}.csv'
log_file = f'log_{today_str}.txt'

session = Session(profile_name='bedrock',region_name='us-east-1')
client = session.client('bedrock-runtime')

def run_claude(prompt_text):
    prompt = f'\n\nHuman: {prompt_text} \n\nAssistant:'
    body = {
        'prompt': prompt,
        'max_tokens_to_sample': 800,
        'temperature': 0.7,
        'top_p': 0.9,
        'top_k': 250
    }

    response = client.invoke_model(
        modelId='anthropic.claude-instant-v1',
        body=json.dumps(body),
        contentType='application/json',
        accept='application/json'
    )
    
    result = json.loads(response['body'].read())
    return result.get('completion','(回答無し)')

with open(input_file,'r',encoding='utf-8') as f:
    reader = csv.reader(f)
    prompts = [row[0] for row in reader if row]

with open(output_file,'w', encoding='utf-8',newline='') as out_f,\
     open(log_file, 'a', encoding='utf-8') as log_f:

    writer = csv.writer(out_f)
    writer.writerow(['質問','回答'])

    for prompt_text in prompts:
        try:
            answer = run_claude(prompt_text)
            writer.writerow([prompt_text,answer])
            print(f'完了：{prompt_text[:20]}... → 回答を保存')
            log_f.write(f'[成功] {prompt_text} → 回答を保存\n')
            time.sleep(1)

        except Exception as e:
            print(f'エラー： {prompt_text[:20]}... → {e}')
            log_f.write(f'[エラー] {prompt_text} → {e}\n')




完了：日本の魅力を教えてください。... → 回答を保存
完了：AWSの主要サービスを教えてください。... → 回答を保存
完了：ClaudeとchatGPTの違いを教え... → 回答を保存
